In [43]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 500)

In [3]:
COLBERTX="/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/validation_queries_retrievals/colbertx_all_znormalized_validation_scores_000_199.tsv"

E5_RUN="tests/e5_docs_000_999_50_sampled_queries_run.txt"

In [4]:
ALL_QUERIES="tests/all_queries.tsv"

In [47]:
CLUEWEB_PARTS_FOLDER="/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/clueweb22-pt_10M_sample_fixed"

In [5]:
colbertx_df = pd.read_csv(COLBERTX, sep="\t")

In [6]:
colbertx_df[colbertx_df['query_id'] == 1]['doc_position'].unique().shape

(1000,)

In [7]:
e5_df = pd.read_csv(E5_RUN, sep=" ", names=["query_id", "Q0", "clueweb_id", "position", "score", "comment"])

In [8]:
e5_df

,query_id,Q0,clueweb_id,position,score,comment
0,1,Q0,clueweb22-pt0000-96-00412_0,1,0.864705,gpt-4-1106-preview
1,1,Q0,clueweb22-pt0001-22-15255_0,2,0.859424,gpt-4-1106-preview
2,1,Q0,clueweb22-pt0001-90-10791_1,3,0.855316,gpt-4-1106-preview
3,1,Q0,clueweb22-pt0002-01-18728_109,4,0.854150,gpt-4-1106-preview
4,1,Q0,clueweb22-pt0001-66-11652_2,5,0.853379,gpt-4-1106-preview
...,...,...,...,...,...,...
49995,199,Q0,clueweb22-pt0001-91-06487_2,996,0.809437,gpt-4-1106-preview
49996,199,Q0,clueweb22-pt0001-02-01819_2,997,0.809434,gpt-4-1106-preview
49997,199,Q0,clueweb22-pt0000-35-19159_2,998,0.809432,gpt-4-1106-preview
49998,199,Q0,clueweb22-pt0000-51-01470_19,999,0.809426,gpt-4-1106-preview


In [9]:
how_many_docs = 1000

result_dfs = []

for group_name, group_df in e5_df.groupby('query_id'):
    
    print("Computing RRF for query={}".format(group_name))
    
    colbertx_query_df = colbertx_df[colbertx_df['query_id'] == group_name].sort_values('score_znorm', ascending=False).iloc[:how_many_docs].reset_index(drop=True)
    
    colbertx_query_df['colbertx_position'] = list(range(1, how_many_docs + 1))
    
    combination_df = colbertx_query_df.merge(group_df, left_on="clueweb_id", right_on="clueweb_id", how='outer').fillna(np.inf)
    
    combination_df['rrf_score_colbertx'] = 1/(60 + combination_df['colbertx_position'])
    combination_df['rrf_score_e5'] = 1/(60 + combination_df['position'])
    
    combination_df['rrf_score'] = combination_df['rrf_score_colbertx'] + combination_df['rrf_score_e5']
    
    result_df = combination_df.sort_values('rrf_score', ascending=False)[['colbertx_id', 'clueweb_id', 'colbertx_position', 'score_znorm', 'position', 'score', 'rrf_score', 'rrf_score_colbertx', 'rrf_score_e5']].copy()
    result_df['query_id'] = group_name
    
    result_df = result_df[['query_id', 'clueweb_id', 'colbertx_id', 'colbertx_position', 'score_znorm', 'position', 'score', 'rrf_score', 'rrf_score_colbertx', 'rrf_score_e5']]
    
    result_df.columns = ['query_id', 'clueweb_id', 'colbertx_id', 'colbertx_position', 'colbertx_score_znorm', 'e5_position', 'e5_score', 'rrf_score', 'rrf_score_colbertx', 'rrf_score_e5']
    
#     result_df['colbertx_id'] = result_df['colbertx_id'].astype(int)
#     result_df['colbertx_position'] = result_df['colbertx_position'].astype(int)
#     result_df['e5_position'] = result_df['e5_position'].astype(int)
    
    result_df['final_index'] = list(range(1, result_df.shape[0] + 1))
    
    result_dfs.append(result_df)

Computing RRF for query=1
Computing RRF for query=2
Computing RRF for query=9
Computing RRF for query=11
Computing RRF for query=13
Computing RRF for query=15
Computing RRF for query=17
Computing RRF for query=20
Computing RRF for query=21
Computing RRF for query=22
Computing RRF for query=24
Computing RRF for query=26
Computing RRF for query=28
Computing RRF for query=47
Computing RRF for query=49
Computing RRF for query=51
Computing RRF for query=54
Computing RRF for query=60
Computing RRF for query=62
Computing RRF for query=64
Computing RRF for query=68
Computing RRF for query=84
Computing RRF for query=95
Computing RRF for query=98
Computing RRF for query=105
Computing RRF for query=113
Computing RRF for query=115
Computing RRF for query=117
Computing RRF for query=126
Computing RRF for query=127
Computing RRF for query=128
Computing RRF for query=136
Computing RRF for query=147
Computing RRF for query=152
Computing RRF for query=153
Computing RRF for query=154
Computing RRF for q

In [10]:
len(result_dfs)

50

In [11]:
final_df = pd.concat(result_dfs).reset_index(drop=True)

In [12]:
final_df

,query_id,clueweb_id,colbertx_id,colbertx_position,colbertx_score_znorm,e5_position,e5_score,rrf_score,rrf_score_colbertx,rrf_score_e5,final_index
0,1,clueweb22-pt0001-22-15255_0,5877599.0,2.0,3.490904,2.0,0.859424,0.032258,0.016129,0.016129,1
1,1,clueweb22-pt0000-86-19982_33,4138910.0,5.0,3.365722,10.0,0.846409,0.029670,0.015385,0.014286,2
2,1,clueweb22-pt0001-66-11652_2,7987527.0,15.0,3.039356,5.0,0.853379,0.028718,0.013333,0.015385,3
3,1,clueweb22-pt0000-34-10889_23,1658352.0,14.0,3.048167,6.0,0.849858,0.028665,0.013514,0.015152,4
4,1,clueweb22-pt0000-77-06884_1,3668516.0,9.0,3.183429,14.0,0.844521,0.028006,0.014493,0.013514,5
...,...,...,...,...,...,...,...,...,...,...,...
92498,199,clueweb22-pt0000-26-19543_91,1291305.0,998.0,2.902421,inf,inf,0.000945,0.000945,0.000000,1603
92499,199,clueweb22-pt0000-51-01470_19,inf,inf,inf,999.0,0.809426,0.000944,0.000000,0.000944,1604
92500,199,clueweb22-pt0000-32-21873_3,1571313.0,999.0,2.902384,inf,inf,0.000944,0.000944,0.000000,1605
92501,199,clueweb22-pt0000-41-17462_6,2006550.0,1000.0,2.902323,inf,inf,0.000943,0.000943,0.000000,1606


### Compare passages returned for a single query

In [13]:
final_df[final_df['query_id'] == 1].head(10)

,query_id,clueweb_id,colbertx_id,colbertx_position,colbertx_score_znorm,e5_position,e5_score,rrf_score,rrf_score_colbertx,rrf_score_e5,final_index
0,1,clueweb22-pt0001-22-15255_0,5877599.0,2.0,3.490904,2.0,0.859424,0.032258,0.016129,0.016129,1
1,1,clueweb22-pt0000-86-19982_33,4138910.0,5.0,3.365722,10.0,0.846409,0.029670,0.015385,0.014286,2
2,1,clueweb22-pt0001-66-11652_2,7987527.0,15.0,3.039356,5.0,0.853379,0.028718,0.013333,0.015385,3
3,1,clueweb22-pt0000-34-10889_23,1658352.0,14.0,3.048167,6.0,0.849858,0.028665,0.013514,0.015152,4
4,1,clueweb22-pt0000-77-06884_1,3668516.0,9.0,3.183429,14.0,0.844521,0.028006,0.014493,0.013514,5
5,1,clueweb22-pt0001-53-03652_0,7347298.0,4.0,3.380061,23.0,0.841374,0.027673,0.015625,0.012048,6
6,1,clueweb22-pt0001-70-06195_8,8157162.0,1.0,3.500292,37.0,0.838730,0.026703,0.016393,0.010309,7
7,1,clueweb22-pt0000-24-15339_2,1191914.0,24.0,2.945330,33.0,0.839504,0.022657,0.011905,0.010753,8
8,1,clueweb22-pt0001-26-13111_1,6065023.0,63.0,2.747783,9.0,0.846435,0.022623,0.008130,0.014493,9
9,1,clueweb22-pt0001-85-15120_6,8849578.0,33.0,2.893970,36.0,0.838852,0.021169,0.010753,0.010417,10


In [14]:
colbertx_df[colbertx_df['query_id'] == 1].sort_values('score_znorm', ascending=False).head(10)

,query_id,colbertx_id,doc_position,doc_score,score_znorm,part,clueweb_id
8,1,8157162,1,28.392147,3.500292,4,clueweb22-pt0001-70-06195_8
184,1,5877599,1,28.366890,3.490904,2,clueweb22-pt0001-22-15255_0
71,1,3917442,1,28.238792,3.408305,1,clueweb22-pt0000-82-13187_0
143,1,7347298,1,28.168346,3.380061,3,clueweb22-pt0001-53-03652_0
1,1,4138910,1,28.135212,3.365722,2,clueweb22-pt0000-86-19982_33
303,1,5877600,2,28.072420,3.331794,2,clueweb22-pt0001-22-15255_1
282,1,2795809,1,27.881601,3.216626,1,clueweb22-pt0000-59-04599_2
371,1,827050,1,27.866114,3.189587,0,clueweb22-pt0000-17-01771_1
379,1,3668516,1,27.819740,3.183429,1,clueweb22-pt0000-77-06884_1
1293,1,6981074,1,27.641541,3.097279,3,clueweb22-pt0001-45-06589_3


In [15]:
e5_df[e5_df['query_id'] == 1].sort_values('score', ascending=False).head(10)

,query_id,Q0,clueweb_id,position,score,comment
0,1,Q0,clueweb22-pt0000-96-00412_0,1,0.864705,gpt-4-1106-preview
1,1,Q0,clueweb22-pt0001-22-15255_0,2,0.859424,gpt-4-1106-preview
2,1,Q0,clueweb22-pt0001-90-10791_1,3,0.855316,gpt-4-1106-preview
3,1,Q0,clueweb22-pt0002-01-18728_109,4,0.854150,gpt-4-1106-preview
4,1,Q0,clueweb22-pt0001-66-11652_2,5,0.853379,gpt-4-1106-preview
5,1,Q0,clueweb22-pt0000-34-10889_23,6,0.849858,gpt-4-1106-preview
6,1,Q0,clueweb22-pt0000-40-08202_0,7,0.848849,gpt-4-1106-preview
7,1,Q0,clueweb22-pt0001-79-01222_35,8,0.847515,gpt-4-1106-preview
8,1,Q0,clueweb22-pt0001-26-13111_1,9,0.846435,gpt-4-1106-preview
9,1,Q0,clueweb22-pt0000-86-19982_33,10,0.846409,gpt-4-1106-preview


### Merge query texts

In [16]:
all_queries_df = pd.read_csv(ALL_QUERIES, sep='\t')

In [17]:
all_queries_df

,query_id,query
0,0,Qual a maior característica da flora brasileira?
1,1,Qual a maior característica da fauna brasileira?
2,2,Por que os países Guiana e Suriname não são filiados a Conmebol?
3,3,Por que a Bolívia tem duas capitais?
4,4,Qual o que significa decolonialidade?
5,5,O que é sistema Swift?
6,6,Por que a seleção uruguaia ostenta 4 estrelas em seu escudo?
7,7,Qual a importância da Semana de Arte Moderna de 1922?
8,8,Por que Johann Sebastian Bach é considerado um revolucionário na história da música?
9,9,quais países europeus seguem o regime monarquista?


In [18]:
final_df = final_df.merge(all_queries_df)

In [19]:
final_df

,query_id,clueweb_id,colbertx_id,colbertx_position,colbertx_score_znorm,e5_position,e5_score,rrf_score,rrf_score_colbertx,rrf_score_e5,final_index,query
0,1,clueweb22-pt0001-22-15255_0,5877599.0,2.0,3.490904,2.0,0.859424,0.032258,0.016129,0.016129,1,Qual a maior característica da fauna brasileira?
1,1,clueweb22-pt0000-86-19982_33,4138910.0,5.0,3.365722,10.0,0.846409,0.029670,0.015385,0.014286,2,Qual a maior característica da fauna brasileira?
2,1,clueweb22-pt0001-66-11652_2,7987527.0,15.0,3.039356,5.0,0.853379,0.028718,0.013333,0.015385,3,Qual a maior característica da fauna brasileira?
3,1,clueweb22-pt0000-34-10889_23,1658352.0,14.0,3.048167,6.0,0.849858,0.028665,0.013514,0.015152,4,Qual a maior característica da fauna brasileira?
4,1,clueweb22-pt0000-77-06884_1,3668516.0,9.0,3.183429,14.0,0.844521,0.028006,0.014493,0.013514,5,Qual a maior característica da fauna brasileira?
...,...,...,...,...,...,...,...,...,...,...,...,...
92498,199,clueweb22-pt0000-26-19543_91,1291305.0,998.0,2.902421,inf,inf,0.000945,0.000945,0.000000,1603,Quais foram os principais artistas do Impressionismo?
92499,199,clueweb22-pt0000-51-01470_19,inf,inf,inf,999.0,0.809426,0.000944,0.000000,0.000944,1604,Quais foram os principais artistas do Impressionismo?
92500,199,clueweb22-pt0000-32-21873_3,1571313.0,999.0,2.902384,inf,inf,0.000944,0.000944,0.000000,1605,Quais foram os principais artistas do Impressionismo?
92501,199,clueweb22-pt0000-41-17462_6,2006550.0,1000.0,2.902323,inf,inf,0.000943,0.000943,0.000000,1606,Quais foram os principais artistas do Impressionismo?


### Save the results

In [20]:
final_df.to_csv("tests/colbertx_e5_RRF_000_999_sampled_50_queries.tsv", sep="\t", index=False)

In [21]:
final_df['Q0'] = "Q0"
final_df['comment'] = "colbertx+e5_RRF"

In [22]:
final_df[['query_id', 'Q0', "clueweb_id", "final_index", "rrf_score", "comment"]].to_csv("tests/colbertx_e5_RRF_000_999_sampled_50_queries_run.txt", header=None, index=False, sep=" ")

### Check already evaluated passages

In [23]:
import pickle

In [24]:
with open("tests/execution_with_reverse_rev_2.1_sampled_50_queries_gpt-4-1106-preview_20240110.pkl", "rb") as input_file:
    logs = pickle.load(input_file)

In [25]:
logs.keys()

dict_keys(['args', 'evaluation_cache', 'reverse_mapping'])

In [26]:
list(logs['evaluation_cache'].keys())[0]

'Onde está localizada a Praça XV de Novembro?_clueweb22-pt0000-44-08794_4'

In [27]:
final_df['query'].unique()

array(['Qual a maior característica da fauna brasileira?',
       'Por que os países Guiana e Suriname não são filiados a Conmebol?',
       'quais países europeus seguem o regime monarquista?',
       'quais os critérios de definição dos monumentos intitulados maravilhas do mundo moderno?',
       'Como o Brasil reagiu a epidemia de AIDS no fim do século XX?',
       'Qual a maior torcida de futebol do Brasil?',
       'Quando se realizou o plebiscito popular para definir o sistema político do Brasil?',
       'Por que a legislação de um país é tão importante?',
       'Qual o nosso atual sistema de governo?',
       'Qual o sistema de governo da África do Sul?',
       'Por que dividir um país em estados?',
       'Como podemos classificar o relevo brasileiro?',
       'Existem vantagens ao definir uma moeda única?',
       'Como transformar uma cidade pacata em um polo turístico?',
       'Quais são os melhores parques nacionais de Portugal?',
       'Qual a melhor cidade para ficar

In [28]:
already_analized = 0

In [30]:
for i, row in final_df.iterrows():
    if (row['query'] + "_" + row['clueweb_id']) in logs['evaluation_cache']:
        print("Already analized...")

        already_analized += 1

Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...


Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...


Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...


Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...


Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...


In [31]:
already_analized

2519

In [33]:
final_10_docs_df = final_df.sort_values(['query_id', 'rrf_score'], ascending=[True, False]).groupby('query_id').head(10)

In [34]:
final_10_docs_df.head(12)

,query_id,clueweb_id,colbertx_id,colbertx_position,colbertx_score_znorm,e5_position,e5_score,rrf_score,rrf_score_colbertx,rrf_score_e5,final_index,query,Q0,comment
0,1,clueweb22-pt0001-22-15255_0,5877599.0,2.0,3.490904,2.0,0.859424,0.032258,0.016129,0.016129,1,Qual a maior característica da fauna brasileira?,Q0,colbertx+e5_RRF
1,1,clueweb22-pt0000-86-19982_33,4138910.0,5.0,3.365722,10.0,0.846409,0.029670,0.015385,0.014286,2,Qual a maior característica da fauna brasileira?,Q0,colbertx+e5_RRF
2,1,clueweb22-pt0001-66-11652_2,7987527.0,15.0,3.039356,5.0,0.853379,0.028718,0.013333,0.015385,3,Qual a maior característica da fauna brasileira?,Q0,colbertx+e5_RRF
3,1,clueweb22-pt0000-34-10889_23,1658352.0,14.0,3.048167,6.0,0.849858,0.028665,0.013514,0.015152,4,Qual a maior característica da fauna brasileira?,Q0,colbertx+e5_RRF
4,1,clueweb22-pt0000-77-06884_1,3668516.0,9.0,3.183429,14.0,0.844521,0.028006,0.014493,0.013514,5,Qual a maior característica da fauna brasileira?,Q0,colbertx+e5_RRF
5,1,clueweb22-pt0001-53-03652_0,7347298.0,4.0,3.380061,23.0,0.841374,0.027673,0.015625,0.012048,6,Qual a maior característica da fauna brasileira?,Q0,colbertx+e5_RRF
6,1,clueweb22-pt0001-70-06195_8,8157162.0,1.0,3.500292,37.0,0.838730,0.026703,0.016393,0.010309,7,Qual a maior característica da fauna brasileira?,Q0,colbertx+e5_RRF
7,1,clueweb22-pt0000-24-15339_2,1191914.0,24.0,2.945330,33.0,0.839504,0.022657,0.011905,0.010753,8,Qual a maior característica da fauna brasileira?,Q0,colbertx+e5_RRF
8,1,clueweb22-pt0001-26-13111_1,6065023.0,63.0,2.747783,9.0,0.846435,0.022623,0.008130,0.014493,9,Qual a maior característica da fauna brasileira?,Q0,colbertx+e5_RRF
9,1,clueweb22-pt0001-85-15120_6,8849578.0,33.0,2.893970,36.0,0.838852,0.021169,0.010753,0.010417,10,Qual a maior característica da fauna brasileira?,Q0,colbertx+e5_RRF


In [35]:
final_10_docs_df.shape

(500, 14)

In [36]:
already_analized = 0

In [37]:
for i, row in final_10_docs_df.iterrows():
    if (row['query'] + "_" + row['clueweb_id']) in logs['evaluation_cache']:
        print("Already analized...")

        already_analized += 1

Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...
Already analized...


In [38]:
already_analized

460

### Add passage texts

In [60]:
id_conversion_df = pd.read_csv(os.path.join(CLUEWEB_PARTS_FOLDER, "clueweb22-pt_colbertx_to_clueweb_id.tsv"), sep="\t", names=['colbertx_id', 'clueweb_id'])

In [61]:
final_10_docs_df = final_10_docs_df.rename(columns={'colbertx_id': 'old_colbertx_id'})

In [62]:
final_10_docs_df = final_10_docs_df.merge(id_conversion_df, left_on='clueweb_id', right_on='clueweb_id')[list(final_10_docs_df.columns) + ['colbertx_id']]

In [63]:
final_10_docs_df

,query_id,clueweb_id,old_colbertx_id,colbertx_position,colbertx_score_znorm,e5_position,e5_score,rrf_score,rrf_score_colbertx,rrf_score_e5,final_index,query,Q0,comment,colbertx_id
0,1,clueweb22-pt0001-22-15255_0,5877599.0,2.0,3.490904,2.0,0.859424,0.032258,0.016129,0.016129,1,Qual a maior característica da fauna brasileira?,Q0,colbertx+e5_RRF,5877599
1,1,clueweb22-pt0000-86-19982_33,4138910.0,5.0,3.365722,10.0,0.846409,0.029670,0.015385,0.014286,2,Qual a maior característica da fauna brasileira?,Q0,colbertx+e5_RRF,4138910
2,1,clueweb22-pt0001-66-11652_2,7987527.0,15.0,3.039356,5.0,0.853379,0.028718,0.013333,0.015385,3,Qual a maior característica da fauna brasileira?,Q0,colbertx+e5_RRF,7987527
3,1,clueweb22-pt0000-34-10889_23,1658352.0,14.0,3.048167,6.0,0.849858,0.028665,0.013514,0.015152,4,Qual a maior característica da fauna brasileira?,Q0,colbertx+e5_RRF,1658352
4,54,clueweb22-pt0000-34-10889_23,1658352.0,2.0,4.307774,8.0,0.883903,0.030835,0.016129,0.014706,1,Quais são os biomas do Brasil?,Q0,colbertx+e5_RRF,1658352
5,1,clueweb22-pt0000-77-06884_1,3668516.0,9.0,3.183429,14.0,0.844521,0.028006,0.014493,0.013514,5,Qual a maior característica da fauna brasileira?,Q0,colbertx+e5_RRF,3668516
6,1,clueweb22-pt0001-53-03652_0,7347298.0,4.0,3.380061,23.0,0.841374,0.027673,0.015625,0.012048,6,Qual a maior característica da fauna brasileira?,Q0,colbertx+e5_RRF,7347298
7,1,clueweb22-pt0001-70-06195_8,8157162.0,1.0,3.500292,37.0,0.838730,0.026703,0.016393,0.010309,7,Qual a maior característica da fauna brasileira?,Q0,colbertx+e5_RRF,8157162
8,1,clueweb22-pt0000-24-15339_2,1191914.0,24.0,2.945330,33.0,0.839504,0.022657,0.011905,0.010753,8,Qual a maior característica da fauna brasileira?,Q0,colbertx+e5_RRF,1191914
9,54,clueweb22-pt0000-24-15339_2,1191914.0,7.0,4.119544,5.0,0.887670,0.030310,0.014925,0.015385,2,Quais são os biomas do Brasil?,Q0,colbertx+e5_RRF,1191914


In [48]:
collection_parts = glob.glob(os.path.join(CLUEWEB_PARTS_FOLDER, "clueweb22-pt_colbertx_0?.tsv"))

In [49]:
collection_parts

['/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/clueweb22-pt_10M_sample_fixed/clueweb22-pt_colbertx_01.tsv',
 '/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/clueweb22-pt_10M_sample_fixed/clueweb22-pt_colbertx_00.tsv',
 '/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/clueweb22-pt_10M_sample_fixed/clueweb22-pt_colbertx_02.tsv',
 '/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/clueweb22-pt_10M_sample_fixed/clueweb22-pt_colbertx_03.tsv',
 '/media/eduseiti/bigdata01/unicamp/ia368v_dd/trabalho_final/clueweb22-pt_10M_sample_fixed/clueweb22-pt_colbertx_04.tsv']

In [64]:
all_parts = []

In [65]:
for i, collection_part in enumerate(collection_parts):
    print("Merging {}...".format(os.path.basename(collection_part)))

    part_df = pd.read_csv(collection_part, sep="\t", header=None, names=['colbertx_id', 'passage'])

    all_parts.append(final_10_docs_df.merge(part_df))

Merging clueweb22-pt_colbertx_01.tsv...
Merging clueweb22-pt_colbertx_00.tsv...
Merging clueweb22-pt_colbertx_02.tsv...
Merging clueweb22-pt_colbertx_03.tsv...
Merging clueweb22-pt_colbertx_04.tsv...


In [67]:
all_parts_df = pd.concat(all_parts)

In [68]:
all_parts_df.head()

,query_id,clueweb_id,old_colbertx_id,colbertx_position,colbertx_score_znorm,e5_position,e5_score,rrf_score,rrf_score_colbertx,rrf_score_e5,final_index,query,Q0,comment,colbertx_id,passage
0,1,clueweb22-pt0000-77-06884_1,3668516.0,9.0,3.183429,14.0,0.844521,0.028006,0.014493,0.013514,5,Qual a maior característica da fauna brasileira?,Q0,colbertx+e5_RRF,3668516,"O tamanho do Brasil e a grande variedade de climas e características e suas regiões são o que torna possível existir no país um número tão alto de espécies. Dentre os biomas brasileiros, os maiores em biodiversidade são a Amazônia, a Mata Atlântica e o Cerrado. Este post é inspirado no livro infantil “Jornada pela Floresta” No livro personalizado Uma Jornada pela Floresta, as crianças são apresentadas de forma divertida e lúdica a vários animais típicos do Brasil: boto-cor-de-rosa, mico-leão-dourado, arara-azul, sucuri, ariranha e outros mais. Logo a seguir, você pode conferir curiosidades sobre cada espécie para compartilhar com os pequenos e aprender mais sobre esses animais que todos nós temos o dever de cuidar e proteger! Boto-cor-de-rosa O boto-cor-de-rosa nasce cinza e muda de cor com o passar do tempo. O boto-cor-de-rosa é um mamífero que vive na água. Ele é parente das baleias e dos golfinhos: a diferença é que ele não vive no mar, mas sim na água doce. Seu habitat são os rios amazônicos do Brasil, da Bolívia, da Venezuela, da Colômbia, do Equador e do Peru."
1,2,clueweb22-pt0000-79-14715_2,3779356.0,21.0,1.459707,26.0,0.845386,0.023974,0.012346,0.011628,5,Por que os países Guiana e Suriname não são filiados a Conmebol?,Q0,colbertx+e5_RRF,3779356,"Devido à forte presença da Igreja Ortodoxa Copta (a igreja cristão nacional do Egito) no país, o calendário tradicional ainda é influente na Etiópia. Pra se ter uma idéia, as celebrações pela chegada do novo milênio na Etiópia só aconteceram em 2007. 4. A GROENLÂNDIA NÃO PODE SE JUNTAR À FIFA PORQUE NÃO HÁ GRAMA NO PAÍS Devido às temperaturas baixas, o solo da Groenlândia está sempre coberto de gelo, o que torna impossível o crescimento de grama e capim suficientes para fazer um campo de futebol que siga às regulamentações da FIFA. Os jogos por lá acontecem na terra, no concreto ou até mesmo no cascalho. (Lembrando que a Groenlândia não é um país independente, ele pertence à Dinamarca). 5. NAURU NÃO POSSUI CAPITAL Embora muitos considerem o distrito de Yaren como capital, por abrigar a maior parte dos escritórios do governo, esse país da Oceania não possui uma capital oficial. 6. TUVALU RECEBEU US$ 50 MILHÕES EM TROCA DOS DIREITOS DO DOMÍNIO “.TV” Isso aconteceu porque o “.tv” é o domínio oficial do país e ao mesmo tempo é popular, já que lembra a palavra “televisão”. E embora pareça uma grande quantia, nos últimos anos o governo de Tuvalu alegou não estar recebendo tanto quanto merece. Confira a lista completa clicando aqui. Siga o blog Road For Two no Facebook."
2,2,clueweb22-pt0000-61-07721_39,2899875.0,289.0,0.322807,8.0,0.850579,0.017571,0.002865,0.014706,10,Por que os países Guiana e Suriname não são filiados a Conmebol?,Q0,colbertx+e5_RRF,2899875,Países Argentina • Bolívia • Brasil • Chile • Colômbia • Equador • Guiana • Paraguai • Peru • Suriname • Uruguai • Venezuela Dependências Ilhas Malvinas (Reino Unido) • Ilhas Geórgia do Sul e Sandwich do Sul (Reino Unido) • Guiana Francesa (França) [Expandir] v d e Organização Mundial do Comércio (OMC) Pessoas Diretora-geral: Ngozi Okonjo-Iweala Ex-directores-gerais: Roberto Azevêdo • Pascal Lamy • Supachai Panitchpakdi • Mike Moore • Renato Ruggiero • Peter Sutherland Vice diretores-gerais: Alejandro Jara • Valentine Rugwabiza • Harsha Singh • Rufus Yerxa Acordos Acordo de Marraquexe • Acordo Geral de Tarifas e Comércio • Agricultura • Aplicação de Medidas Sanitárias e Fitossanitárias • Barreiras Técnicas ao Comércio • Comércio de Serviços • Declaração de Doha • Medidas de Investimento Relacionadas ao Comércio • Procuração de Governo • Tecnologia da Informação • TRIPs P

In [69]:
all_parts_df.shape

(500, 16)

In [70]:
all_parts_df.rename(columns={'clueweb_id': 'passage_id'}).to_csv(os.path.join("tests", "colbertx_e5_RRF_000_009_sampled_50_queries.tsv"), sep="\t", index=False)